<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/word-lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import matplotlib.pyplot as plt
import os

usefull links:

https://www.tensorflow.org/tutorials/text/text_generation

https://github.com/petrosDemetrakopoulos/RNN-Beatles-lyrics-generator/blob/master/model.py

# Data

In [0]:
def corpusToList(corpus):
  corpusList = [w for w in corpus.split(' ')] 
  corpusList = [i for i in corpusList if i]
  return corpusList

In [4]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname2.txt", url)

text = open(filepath, 'rb').read().decode(encoding='utf-8')
text = text.replace("\t"," \t ").replace("\n", " \n ")
corpusList = [w for w in text.split(' ')] 
corpus_words = [i for i in corpusList if i]
# corpus_words = corpusToList(text) 
map(str.strip, corpus_words) #trim words

In [6]:
vocab = sorted(set(corpus_words))
print('Corpus length (in words):', len(corpus_words))
print('Unique words in corpus: {}'.format(len(vocab)))
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpus_words])

Corpus length (in words): 661330
Unique words in corpus: 19780


In [0]:
seqLength = 20
examples_per_epoch = len(corpus_words)//(seqLength + 1)
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)
sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True)
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequencesOfWords.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 100
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Model (GRU)

In [0]:
def create_model_gru(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [15]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

gru_model = create_model_gru(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

gru_model.summary()

gru_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = gru_model.fit(dataset, epochs=20)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           5063680   
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 19780)         20274500  
Total params: 29,276,484
Trainable params: 29,276,484
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
492/492 [==============================] - 46s 94ms/step - loss: 6.4823
Epoch 2/20
492/492 [==============================] - 46s 94ms/step - loss: 5.6372
Epoch 3/20
492/492 [==============================] - 46s 94ms/step - loss: 4.9428
Epoch 4/20
492/492 [==============================] - 46s 94ms/step - loss: 4.5214
Epoch 5/20
492/492 [====================

In [19]:
main_gru_model = create_model_gru(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
main_gru_model.set_weights(gru_model.get_weights())
main_gru_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            5063680   
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_6 (Dense)              (1, None, 19780)          20274500  
Total params: 29,276,484
Trainable params: 29,276,484
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 200
  start_string_list =  [w for w in start_string.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2words[predicted_id])
  return (start_string + ' '.join(text_generated))

In [33]:
print(generate_text(main_model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1))

که ایران چوباغیست خرم بهار	 گهی مهربان بودی و گاهی کلاه 
 چو آن دیده‌بانان ایران سپاه 	 نگون اندر آمد ز پهلو براه 
 چو هومان ازان جایگاه نبرد 	 ز گودرز کمتر گریزان ز باد 
 بنزدیک پیران و هومان گیو 	 نشستند و گفتند کین رای دیگر نگهدار من 
 چنین داد پاسخ ورا پیرمرد 	 همیدون همیدون ز لشکر سوار 
 ز لشکر سخنگوی مردی هزار 	 سواران و شیران خنجرگزار 
 بقلب اندر آمد سخن راندند 	 همه جان یک بادگر درد را تنگ دارند کین خواستن 	 بکینه میان دو رویه جگر 
 تن خسته بودند گردان اسیر 	 روان‌ها همه باد و یزدان‌پرست 
 ز هر گونه بسیار و بنشاندش 	 چو پروین شدش سوی ایشان هنر 
 چو گودرز گودرز پیر و پیکار چیست 
 ز بخشیدن شوم گر ترا 	 گر اینست پیمان تو من نبرد 
 پی سال دیدی ز کار سپاه 	 فراز آوریدند پیلی همیدون بدست 
 که گودرز را جز برین گونه فرمود شاه 	 بفرمود کردن همه سربسر گرد کرد 
 بجویید درباغ بگشاد بیژن پیام 	 سخن چون بباید پرستار جنگ 
 روان مسیحا پر از رنگ و بوی 	 چنان خوش که رفتن ز بیچارگیست 	 ز بهرام


In [0]:
model.save("word_gru.h5")

# Model (LSTM)

In [0]:
def create_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [35]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
lstm_model.summary()
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = lstm_model.fit(dataset, epochs=20)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 256)           5063680   
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense_7 (Dense)              (64, None, 19780)         20274500  
Total params: 30,585,156
Trainable params: 30,585,156
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
492/492 [==============================] - 55s 111ms/step - loss: 6.1975 - accuracy: 0.1123
Epoch 2/20
492/492 [==============================] - 54s 110ms/step - loss: 5.4306 - accuracy: 0.1791
Epoch 3/20
492/492 [==============================] - 54s 111ms/step - loss: 5.0112 - accuracy: 0.2265
Epoch 4/20
492/492 [==============================] - 54s 111ms

In [36]:
main_lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
main_lstm_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 256)            5063680   
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_8 (Dense)              (1, None, 19780)          20274500  
Total params: 30,585,156
Trainable params: 30,585,156
Non-trainable params: 0
_________________________________________________________________


In [43]:
print(generate_text(main_lstm_model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1))

که ایران چوباغیست خرم بهار
 فروتر سر از جای برخاستند 	 بدیوان شاه از ایشان نکرد 
 به پیلان برآمد سپه برگرفت 	 ازیشان سخنها فراوان دلیر 
 ازان پس رستم رای‌زن هرک آمد سپاه 	 از آن روزبانان مردم‌کشان 	 که ای شهریار جهان سربسر 
 کجا سرو کشمرش گیرد بجنگ 	 بیامد بدلش اندر آمد چو نیل 
 دو دست از پس پشت بستش چو کوه 	 بپیران نیارست شد ترجمان 
 ببالا برآمد بروی نبرد 	 شب و روز با تن شدند انجمن 
 بدو بر ما یکی نامدار 	 دلیر و پیاده شدش نیو را 
 ابا ده هزار آزموده سران 	 هشیوار و کردند تن سربسر هدیه گشت 
 سراسر همه جامه و سیم 	 پر از کینه و زرمساز آمدند 
 ببارید خون دو رخ لاجورد 	 هم آنگه بغلتید بر خیره روی 
 چپ و بوم هندوستان 	 گروگان بدست تو دست آورد 
 سرش را بفتراک شبرنگ بست 
 بشد بیژن گیو تا بود و بس 	 خروشان ازیشان و مردان نیو 
 جدا زیر پیلان و خود برنشست 	 بدل پر ز کینه دلی پر ز گرد 
 برآمد برین کینه درویش نیز 	 ز پیکارشان دل تو بیگانه نیست 
 یکی جنگ با پیل


In [0]:
model.save("word_lstm.h5")